In [ ]:
effnet_path = '../input/pytorch-efficientnet'
import sys,glob
sys.path.append(effnet_path)
import librosa
import cv2,os
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
from pathlib import Path
import os,random
import matplotlib.pyplot as plt
import gc
import matplotlib.image as immg
from tqdm.notebook import tqdm
import albumentations
import torch
import torch.nn as nn
from torch.nn import functional as F
from efficientnet_pytorch import EfficientNet
from fastai.vision.all import *

In [ ]:
!pip install -q timm

In [ ]:
import warnings,os,gc,sys
warnings.filterwarnings('ignore')

In [ ]:
os.listdir('../input/testg2net-audio-wave-data')

In [ ]:
os.listdir('../input/train-g2net-audio-wave-data')

In [ ]:
os.listdir('../input/band-pass-traintest-g2net-audio-wave-data')

In [ ]:
os.mkdir('/kaggle/working/train/')

In [ ]:
!unzip -qq '../input/band-pass-traintest-g2net-audio-wave-data/TrainG2NET.zip' -d '/kaggle/working/train/'

In [ ]:
os.mkdir('/kaggle/working/test/')

In [ ]:
!unzip -qq '../input/band-pass-traintest-g2net-audio-wave-data/TestG2NET.zip' -d '/kaggle/working/test/'

In [ ]:
trn_df = pd.read_csv('../input/train-g2net-audio-wave-data/g2net_train_kfold.csv',usecols=['id','target','kfold'])

In [ ]:
trn_df.head()

In [ ]:
img_path = '/kaggle/working'

In [ ]:
FOLD = 1

In [ ]:
trn_df1 =  trn_df[trn_df.kfold<5].reset_index(drop=True)
val_idxs1 = trn_df1[trn_df1.kfold==FOLD].index

In [ ]:
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [ ]:
dls = ImageDataLoaders.from_df(trn_df1, path=img_path, fn_col = 'id', 
                               folder='train',label_col='target',suff='.png',bs=128,
                               item_tfms=None,batch_tfms=batch_tfms,val_idxs=val_idxs1)

In [ ]:
dls.show_batch(figsize=(20,6))

In [ ]:
len(dls.train_ds),len(dls.valid_ds)

In [ ]:
import sklearn.metrics as sklm

In [ ]:
def rocauc(preds, targs):
    return sklm.roc_auc_score(targs, preds[:,1])

In [ ]:
metrics = [AccumMetric(rocauc, flatten=False)]

In [ ]:
#learn.lr_find()

In [ ]:
import timm

In [ ]:
all_pretrained_models_available = timm.list_models(pretrained=True)

In [ ]:
# np.array(all_pretrained_models_available)

In [ ]:
gc.collect()

In [ ]:
net = timm.create_model('resnest50d',pretrained=True)

In [ ]:
res = nn.Sequential(net,
                    nn.Dropout(0.1),
                    nn.Linear(1000,2))

In [ ]:
# learn = cnn_learner(dls, resnet50,
#                     loss_func=LabelSmoothingCrossEntropyFlat(),
#                     metrics = [RocAucBinary()],
#                     model_dir='/kaggle/working').to_fp16()

In [ ]:
effnet = EfficientNet.from_pretrained('efficientnet-b0',num_classes=dls.c)
learn = Learner(dls, effnet,
                loss_func = CrossEntropyLossFlat(), 
                metrics=[RocAucBinary()],
                model_dir='/kaggle/working/').to_fp16()

In [ ]:
cb1 = SaveModelCallback(monitor='roc_auc_score',fname='best_model_band_pass',comp=np.greater) # Callbacks
cb2 = ReduceLROnPlateau(monitor='roc_auc_score', min_delta=0.01, patience=2,factor=0.2)
learn.fit_one_cycle(10, 1e-3, cbs = [cb1,cb2])

In [ ]:
learn.load('best_model');

In [ ]:
# val_df = trn_df[trn_df.kfold==FOLD].copy()
# val_data_path = val_df['id'].apply(lambda x:'/kaggle/working/train/'+str(x)+'.png').values
# val_dl = learn.dls.test_dl(val_data_path)
# val_prds = learn.get_preds(dl = val_dl)

In [ ]:
# val_df['preds'] = val_prds[0].numpy()[:,1]

In [ ]:
# sklm.roc_auc_score(val_df.target,val_df.preds)

In [ ]:
# val_df.to_csv('valid_prds_resnest50_bandpass.csv',index=False)

In [ ]:
sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
sub.head()

In [ ]:
test_data_path = sub['id'].apply(lambda x:'/kaggle/working/test/'+str(x)+'.png').values
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.get_preds(dl = tst_dl)

In [ ]:
ts_preds = predictions[0].numpy()

In [ ]:
sub['target'] = ts_preds[:,1]
sub.to_csv('submissiong2net4.csv',index=False)

In [ ]:
!rm -r '/kaggle/working/test/'

In [ ]:
!rm -r '/kaggle/working/train/'